# Handling Missing Time Series Data

<style>
    table td {
        vertical-align: middle;
        position: relative;
    }
    table td p {
        display: inline-block;
        vertical-align: middle;
    }
    .center {
        text-align: center;
    }
    .justify {
        display: flex;
        justify-content: space-between;
        align-items: center;
    }
    .green {
        background-color: rgba(144, 238, 144, 0.2);
    }
</style>

## Introduction

TODO

## Causes / Reasons

Some examples:

| <div class="center">Reason</div> | <div class="center">Comment</div>                                                           | <div class="center">Score</div>                                                     |
| -------------------------------- | ------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------- |
|                                  |                                                                                             | <div class="justify"><span>Random</span><span>-</span><span>Systematic</span></div> |
| Sensor failure                   | Data could not be retrieved or kept                                                         | <table><tr><td>⬜</td><td>⬜</td><td>❎</td><td>⬜</td><td>⬜</td></tr></table>          |
| Not applicable                   | If not children, there is no age of your children                                           | <table><tr><td>⬜</td><td>⬜</td><td>⬜</td><td>⬜</td><td>❎</td></tr></table>          |
| Publication lag                  | Sydney market volumes always come 3 months delayed because of complicated gathering process | <table><tr><td>❎</td><td>⬜</td><td>⬜</td><td>⬜</td><td>⬜</td></tr></table>          |
| Drop in/out                      | Categories (dis)appeared at some point in time                                              | <table><tr><td>⬜</td><td>⬜</td><td>⬜</td><td>⬜</td><td>❎</td></tr></table>          |
| Label change                     | Split of a category in to two                                                               | <table><tr><td>⬜</td><td>⬜</td><td>⬜</td><td>⬜</td><td>❎</td></tr></table>          |
| Intentional                      | Intentionally filtered outlier (wrong data suspect)                                         | <table><tr><td>⬜</td><td>⬜</td><td>❎</td><td>⬜</td><td>⬜</td></tr></table>          |
| Refusal                          | Optional data not provided or refused to answer                                             | <table><tr><td>⬜</td><td>⬜</td><td>⬜</td><td>❎</td><td>⬜</td></tr></table>          |
| Table joins                      | Missing data is generated by table joins in SQL                                             | <table><tr><td>⬜</td><td>⬜</td><td>⬜</td><td>❎</td><td>⬜</td></tr></table>          |

### Missing Completely at Random (MCAR)

Mechanisms/examples:

- Reporting lag of last 3 months
- Our pipeline crashed because of unrelated reasons

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td>1.7</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td class="center">
            <!-- <div class="center">Completely<br>random<br>➡️</div> -->
            Completely<br>random<br>➡️
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>&nbsp;</td></tr>
                <tr><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-04</td><td>&nbsp;</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td>1.7</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>&nbsp;</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

### Missing at Random (MAR)

Mechanisms/examples:

- Airlines have a different level of 'completeness' of flight event data
- Older trucks are not yet all equipped with GPS sensors (correlation with capacity, CO2 emissions, etc)
- Shipment not detected as arrived because truck GPS did not hit the geofence properly (location specific problem)

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td>1.7</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">y-values above<br>10.0 have a 50%<br>chance of not<br>being reported<br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td></tr>
                <tr><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

### Missing Not at Random (MNAR)

Mechanisms/examples:

- Old vessel schedules are purged in shipping systems if they have no corresponding shipment.
- Not all airlines deliver US flight events.
- An arrival even in Sydney airport is missing if it arrived at a nearby airport and was trucked to Sydney (road feeder service), plus the suspected handling time is longer in this situation.
- Dates with no volume will be missed in SQL GROUP BY query result.
- Unfinished shipments have no delivery date.
- Shipment rate index is missing in Chinese New Year week.
- GPS sensor turns off when truck is not moving.
- GPS sensor turns off when truck is in tunnel.

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td>1.7</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Some system<br>issues causes<br>values above 11.0<br>to not be reported<br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td><td>&nbsp;</td></tr>
            </table>
        </td>
    </tr>
</table>

### Set up

Before we begin, it is necessary to set up our environment and create a sample data set with missing values. Click on "Show code" to see the full code for all sections on this page. Click "No code" at any time to hide the code.

#### No code

#### Show code

For this guide, we will use a number of libraries to show the functionality. The important ones are:

- [`numpy`] and [`pandas`] for data manipulation
- [`synthetic_data_generators`] to create sample time series data
- [`plotly`] for visualisation
- [`pmdarima`] for ARIMA modelling
- [`sklearn`] for machine learning models
- [`tqdm`] for progress bars

In [1]:
# StdLib Imports
import warnings
from datetime import datetime
from functools import partial
from typing import Literal

# Third Party Imports
import numpy as np
import pandas as pd
from pandas.errors import PerformanceWarning
from plotly import express as px, graph_objects as go, io as pio
from pmdarima import auto_arima
from sklearn.ensemble import RandomForestRegressor
from sklearn.exceptions import DataConversionWarning
from sklearn.metrics import mean_absolute_percentage_error as mape
from synthetic_data_generators.time_series import TimeSeriesGenerator
from tqdm import tqdm

After importing the necessary libraries, we can set up some global settings which will be used throughout the guide. This includes setting a random seed for reproducibility (`RANDOM_SEED`), defining the number of periods for our time series data (`NUM_PERIODS`), and configuring Plotly's default template for visualizations (`pio.templates.default`). We also instantiate the `TimeSeriesGenerator()` class, assigning a constant seed for reproducibility (`TSG`). Additionally, we will suppress certain warnings to keep the output clean.

In [2]:
# Constants, Settings, Instantiations
RANDOM_SEED = 42
NUM_PERIODS = 1096
pio.templates.default = "simple_white+gridon"
TSG = TimeSeriesGenerator(seed=RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=PerformanceWarning)

In this guide, we will constantly be plotting our data to visualise the effects of missing data handling techniques. Therefore, we define a single helper function `plot_data()` that we can reuse for convenience and consistent formatting. It takes in a DataFrame and various parameters to create a line plot comparing the original data with missing values and the filled data. The function also allows for customization of titles, subtitles, output file saving, and whether to show or return the figure.

In [3]:
def plot_data(
    data: pd.DataFrame,
    date_col: str,
    missing_col: str,
    fill_col: str,
    title: str,
    subtitle: str | None = None,
    output_file: str | None = None,
    show_or_return: Literal["show", "return"] = "show",
) -> go.Figure | None:
    fig: go.Figure = (
        px.line(title=f"{title}<br><sup>{subtitle}</sup>" if subtitle else title)
        .add_scatter(
            name="filled",
            x=data[date_col],
            y=data[fill_col],
            mode="lines+markers",
            line_color="crimson",
            line_width=1,
            marker_size=4,
        )
        .add_scatter(
            name="original",
            x=data[date_col],
            y=data[missing_col],
            mode="lines+markers",
            line_color="cornflowerblue",
        )
        .update_layout(
            xaxis_title="Date",
            yaxis_title="Value",
            legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="left", x=0, traceorder="reversed"),
            xaxis_range=[
                data[date_col].min() - pd.offsets.Day(3),
                data[date_col].max() + pd.offsets.Day(3),
            ],
            title=dict(
                x=0.5,
                xanchor="center",
                yanchor="top",
            ),
        )
    )

    if output_file:
        fig.write_html(output_file, include_plotlyjs="cdn")

    if show_or_return == "show":
        fig.show()
    elif show_or_return == "return":
        return fig
    else:
        raise ValueError(f"Invalid value for `show_or_return`: '{show_or_return}'. Must be either: 'show' or 'return'")

Now that we have our environment set up, we can create a sample time series data set with missing values. We will generate seasonal data with a yearly seasonality pattern and then randomly remove 50% of the data points to simulate missing data.

In [5]:
# Set data parameters
start_date = datetime(2023, 1, 1)
NUM_PERIODS = 365
interpolation_nodes: tuple[list[int], ...] = ([0, 160], [7, 160], [14, 160], [34, 160])
level_breaks: list[list[int]] = []
randomwalk_scale: float = 0
season_eff: float = 0.7
noise_scale: float = 15
season_conf: dict[str, int | str] = {
    "style": "sin",
    "period_length": 28 * 6,
    "start_index": 2,
    "amplitude": 2,
}

# Build data set
data: pd.DataFrame = (
    TSG.create_time_series(
        start_date=start_date,
        n_periods=NUM_PERIODS,
        interpolation_nodes=interpolation_nodes,
        level_breaks=level_breaks,
        randomwalk_scale=randomwalk_scale,
        season_conf=season_conf,
        season_eff=season_eff,
        noise_scale=noise_scale,
        seed=RANDOM_SEED,
    )
    .assign(
        Missing=lambda df: np.where(
            df.index.isin(
                np.random.default_rng(seed=RANDOM_SEED).choice(
                    df.index,
                    size=len(df) // 2,
                    replace=False,
                )
            ),
            np.nan,
            df["Value"],
        ),
    )
    .reset_index()
)

When we inspect the generated data, we can see the number of missing values in the "Missing" column and a preview of the first 10 rows.

In [ ]:
# Check data
print(data.isna().sum().to_frame("Num Missing").to_markdown())
print(data.head(10).to_markdown())

|         |   Missing Values |
|:--------|-----------------:|
| index   |                0 |
| Date    |                0 |
| Value   |                0 |
| Missing |              182 |
|    | index               | Date                |   Value |   Missing |
|---:|:--------------------|:--------------------|--------:|----------:|
|  0 | 2023-01-01 00:00:00 | 2023-01-01 00:00:00 | 245.059 |   245.059 |
|  1 | 2023-01-02 00:00:00 | 2023-01-02 00:00:00 | 196.839 |   196.839 |
|  2 | 2023-01-03 00:00:00 | 2023-01-03 00:00:00 | 191.966 |   nan     |
|  3 | 2023-01-04 00:00:00 | 2023-01-04 00:00:00 | 208.287 |   208.287 |
|  4 | 2023-01-05 00:00:00 | 2023-01-05 00:00:00 | 216.724 |   216.724 |
|  5 | 2023-01-06 00:00:00 | 2023-01-06 00:00:00 | 197.463 |   197.463 |
|  6 | 2023-01-07 00:00:00 | 2023-01-07 00:00:00 | 217.824 |   217.824 |
|  7 | 2023-01-08 00:00:00 | 2023-01-08 00:00:00 | 205.426 |   205.426 |
|  8 | 2023-01-09 00:00:00 | 2023-01-09 00:00:00 | 201.368 |   nan     |
|  9 | 2023

|         | Missing Values |
| :------ | -------------: |
| index   |              0 |
| Date    |              0 |
| Value   |              0 |
| Missing |            182 |

|      | index               | Date                |   Value | Missing |
| ---: | :------------------ | :------------------ | ------: | ------: |
|    0 | 2023-01-01 00:00:00 | 2023-01-01 00:00:00 | 245.059 | 245.059 |
|    1 | 2023-01-02 00:00:00 | 2023-01-02 00:00:00 | 196.839 | 196.839 |
|    2 | 2023-01-03 00:00:00 | 2023-01-03 00:00:00 | 191.966 |     nan |
|    3 | 2023-01-04 00:00:00 | 2023-01-04 00:00:00 | 208.287 | 208.287 |
|    4 | 2023-01-05 00:00:00 | 2023-01-05 00:00:00 | 216.724 | 216.724 |
|    5 | 2023-01-06 00:00:00 | 2023-01-06 00:00:00 | 197.463 | 197.463 |
|    6 | 2023-01-07 00:00:00 | 2023-01-07 00:00:00 | 217.824 | 217.824 |
|    7 | 2023-01-08 00:00:00 | 2023-01-08 00:00:00 | 205.426 | 205.426 |
|    8 | 2023-01-09 00:00:00 | 2023-01-09 00:00:00 | 201.368 |     nan |
|    9 | 2023-01-10 00:00:00 | 2023-01-10 00:00:00 | 192.668 |     nan |

We can also visualise the data using our `plot_data()` function, which shows the seasonal pattern along with the missing data points.

In [ ]:
# Plot data
plot_data(
    data=data,
    date_col="Date",
    missing_col="Missing",
    fill_col="Value",
    title="Seasonal Data - With missing data points",
    subtitle="(using synthetic data)",
    output_file="./images/00_seasonal_data_with_missing.html",
)

As you can see with the below plot, the data exhibits a clear seasonal pattern, but there are several missing data points scattered throughout the time series.


## Dealing with Missing Data

### Dropping

Advantages:

- Easy
- Does not create 'fake' data
- Preserves base statistics and correlation between variables

Disadvantages:

- Excessive data loss for multivariate data
- Can create big gaps in your data
- Not suitable if you already have a very small data set
- Can cause sampling bias

When to use:

- Certain features are missing a large percentage of their data
- There is a substantial structural change in the data, which is causing your models to become unstable

#### Dropping Observations

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>&nbsp;</td><td>&nbsp;</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td>1.7</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Dropping the first<br>three rows<br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td>1.7</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

#### Dropping Features

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td>1.7</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Dropping the last<br>column <b>x2</b></b><br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td></tr>
            </table>
        </td>
    </tr>
</table>

### Recording

Advantages:

- Retains the models flexibility for how to fit the missing values
- We can see the effect of the missing data in the estimated parameters

Disadvantages:

- All missing values are assigned the same effect size in linear models (eg. ARIMA)
- Not applicable for the target variable

When to use:

- You do not want data loss
- Same effect size for all missing data is okay or the model is flexible enough to handle it (eg. tree-based models)

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Something<br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th><th>x1_m</th><th>x2_m</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td class="green">0</td><td>1.2</td><td class="green">1</td><td class="green">0</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td><td class="green">0</td><td class="green">0</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td><td class="green">0</td><td class="green">0</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td><td class="green">0</td><td class="green">0</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td class="green">0</td><td class="green">0</td><td>1</td><td class="green">1</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td><td class="green">0</td><td class="green">0</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td><td class="green">0</td><td class="green">0</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>16.4</td><td>0.8</td><td class="green">0</td><td class="green">0</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td class="green">0</td><td>0.9</td><td class="green">1</td><td class="green">0</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td><td class="green">0</td><td class="green">0</td></tr>
            </table>
        </td>
    </tr>
</table>

### Filling using Random Distribution

Eg. random sample from a normal distribution created for the existing data in the specific feature.

Advantages:

- Preserves the base statistics in the data set (mean, variances, etc)
- Applicable also for target variables

Disadvantages:

- But also preserves potential filter bias
- Destroys correlation between variables
- Model cannot distinguish real and imputed data

When to use:

- Other methods are not suitable in your circumstances

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Something</b><br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td class="green">49.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td class="green">19.9</td><td class="green">1.1</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td class="green">22.3</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

#### No code

#### Show code

In [ ]:
### Do fill ----
nml: np.ndarray = np.random.default_rng(seed=42).normal(
    loc=data["Missing"].mean(),
    scale=data["Missing"].std(),
    size=len(data),
)
data_random: pd.DataFrame = data.copy().assign(
    Fill=lambda df: np.where(
        df["Missing"].isna(),
        nml,
        df["Missing"],
    )
)

In [ ]:
### Plot data ----
score_random: float = mape(data_random[["Value"]], data_random[["Fill"]]) * 100
plot_data(
    data=data_random,
    date_col="Date",
    missing_col="Missing",
    fill_col="Fill",
    title="Filling using Random Distribution",
    subtitle=f"MAPE={score_random:.2f}%",
    output_file="./images/01_filling_using_random_distribution.html",
)

### Filling using Feed-Forward

Eg. take the most recent value, and feed it forward to fill the gaps.

Advantages:

- Easy
- Often used as a naïve benchmark because it is super default

Disadvantages:

- Lacks any deeper logical understanding of the data
- Does not account for trends or seasonality, etc

When to use:

- Good as a benchmark or dirty hack only

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Something</b><br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td class="green"><code>NaN</code></td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td class="green">14.4</td><td class="green">0.7</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td class="green">19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

#### No code

#### Show code

In [ ]:
### Do fill ----
data_ffill: pd.DataFrame = data.assign(
    Fill=lambda df: df["Missing"].ffill(),
)

In [ ]:
### Plot data ----
score_ffill: float = mape(data_ffill[["Value"]], data_ffill[["Fill"]]) * 100
plot_data(
    data=data_ffill,
    date_col="Date",
    missing_col="Missing",
    fill_col="Fill",
    title="Filling using Feed-Forward",
    subtitle=f"MAPE={score_ffill:.2f}%",
    output_file="./images/02_filling_using_feed_forward.html",
)

### Filling using Imputation ($σ$ or $x~$)

A statistical method of filling missing values which goes ideally beyond plain 'prediction' of the missing values. Imputation tries to preserve ALL statistical properties of the original (unknown) data, including means, variances, etc., including the noise level.

Advantages:

- Easy to calculate and input mean (σ) or median (x~) values
- Best option to create a filled dataset without any bias for arbitrary statistical analysis later on

Disadvantages:

- Inserted values are not close to the real ones
- Reduction of sample variance
- Complicated, no time-series specific solution is available
- Not very common in machine learning, it is used more in the statistics domain

When to use:

- Other methods are not suitable in your circumstances
- You don't know what kind of analysis will be done on the filled data set later
- You look for a best-in-class solution for filling missing values

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Something</b><br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td class="green">23.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td class="green">23.2</td><td class="green">1.0</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td class="green">23.2</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

#### No code

#### Show code

In [ ]:
### Do fill ----
data_stats: pd.DataFrame = data.assign(
    Fill=lambda df: np.where(
        df["Missing"].isna(),
        df["Missing"].mean(),
        df["Missing"],
    ),
)

In [ ]:
### Plot data ----
score_stats: float = mape(data_stats[["Value"]], data_stats[["Fill"]]) * 100
plot_data(
    data=data_stats,
    date_col="Date",
    missing_col="Missing",
    fill_col="Fill",
    title="Filling using Imputation (average value)",
    subtitle=f"MAPE={score_stats:.2f}%",
    output_file="./images/03_filling_using_imputation.html",
)

### Filling using Interpolation

Unlike statistical prediction (curve fitting), interpolation is a numerical method to overlay a curve into known variables (nodes) such that the curve hits the known values exactly and approximates what happens in between. This makes sense only if there is a continuous trend between the known nodes. You can use methods such as linear interpolation, polynomial, splines, etc.

The difference between imputation and interpolation is that the latter does not try to preserve the statistical properties of the original data, but rather it is a mathematical approach to estimate the missing values based on the known ones (usually by creating a straight-line between known data points). Interpolation is univariate, meaning it only uses the values of the variable itself to estimate the missing values, without considering other variables, and it assumes continuity in the underlying data. Interpolation is used frequently in time series analysis to fill in gaps in data, especially when the data is expected to follow a certain trend or pattern over time.

Advantages:

- Follows the local trend
- Better than feed-forward

Disadvantages:

- Approach is univariate (that is, it does not take in to account exogeneous variables, like holiday information, etc)
- Resembles a trend rather than a seasonality or autocorrelation

When to use:

- You want something not extremely trivial, but is still readily available. For example, Pandas has this method already implemented.
- A good trend fit seems most important.

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>&nbsp;</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Something</b><br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td class="green">31</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td class="green">2.0</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td class="green">1.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>20.1</td><td class="green">1.4</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td class="green">19.3</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td class="green">19.7</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

#### No code

#### Show code

In [ ]:
### Do fill ----
data_interpolation: pd.DataFrame = data.assign(
    Fill=lambda df: df["Missing"].interpolate(),
)

In [ ]:
### Plot data ----
score_interpolation: float = mape(data_interpolation[["Value"]], data_interpolation[["Fill"]]) * 100
plot_data(
    data=data_interpolation,
    date_col="Date",
    missing_col="Missing",
    fill_col="Fill",
    title="Filling using Interpolation",
    subtitle=f"MAPE={score_interpolation:.2f}%",
    output_file="./images/04_filling_using_interpolation.html",
)

### Filling using Time-Series Prediction

Eg. run a forecasting algorithm (like ARIMA) to 'predict' the missing future values.

Advantages:

- Promises a better fit
- Can better reconstruct non-random missingness
- Applicable also for target variable

Disadvantages:

- Only useful if the time series is actually predictable
- Don't forget the Münchhausen trilemma[^munchhausen-trilemma] of trying to create a forecast from predicted data
- Predicted values are filled in 'without noise'; that is, without 'variance biase'

When to use:

- The variables have strong time series properties and strong correlations among each other so that the prediction approach is powerful and precise

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>&nbsp;</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Something</b><br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>18.2</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td class="green">16.6</td><td class="green">0.9</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td class="green">9.3</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td class="green">19.7</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

#### No code

#### Show code

In [ ]:
### Do fill ----
data_forecast: pd.DataFrame = data.copy().assign(Fill=data["Missing"])
indexes_of_missing: list[int] = data_forecast[data_forecast["Fill"].isna()].index.to_list()
for idx in tqdm(indexes_of_missing):
    tmp: pd.DataFrame = data_forecast.loc[:idx]
    fcst_values: np.ndarray = tmp[["Fill"]].values[:-1]
    tmp_modl = auto_arima(
        fcst_values if len(fcst_values) > 2 else np.append(fcst_values[1], fcst_values),
        random_state=RANDOM_SEED,
        seasonal=True,
        stepwise=True,
        error_action="ignore",
    )
    fcst = tmp_modl.predict(n_periods=1, return_conf_int=False)
    data_forecast.loc[idx, "Fill"] = fcst[0]

In [ ]:
### Plot data ----
score_forecast: float = mape(data_forecast["Value"], data_forecast["Fill"]) * 100
plot_data(
    data=data_forecast,
    date_col="Date",
    missing_col="Missing",
    fill_col="Fill",
    title="Filling using Time-Series Forecasting (ARIMA)",
    subtitle=f"MAPE={score_forecast:.2f}%",
    output_file="./images/05_filling_using_arima_forecasting.html",
)

### Filling using Algorithmic Prediction (Classification & Regression)

Eg. run a prediction algorithm (whether it be a classification or regression problem) over missing data in the predictor features.

Advantages:

- Promises a better fit
- Can better reconstruct non-random missingness
- Applicable also for target variable

Disadvantages:

- Münchhausen trilemma[^munchhausen-trilemma] also applies
- Predicted values are filled in 'without noise'; that is, without 'variance biase'

When to use:

- Other methods are not suitable in your circumstances

**The Core Problem**

Classical ML algorithms like Random Forest, XGBoost, or Linear Regression treat each row as i.i.d. (independent and identically distributed). Time series violates this assumption because:

- **Temporal autocorrelation**: Today's value depends on yesterday's
- **Trend**: Values systematically increase/decrease over time
- **Seasonality**: Patterns repeat at regular intervals
- **Order matters**: Shuffling rows destroys information

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Something</b><br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td class="green">&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td class="green">&nbsp;</td><td class="green">&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td class="green">&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

#### No code

#### Show code

In [ ]:
def build_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    assert "Date" in df.columns, "DataFrame must contain 'Date' column"
    tmp: pd.DataFrame = df.copy()
    return tmp.assign(
        Year=lambda df: df["Date"].dt.year,
        MonthOfYear=lambda df: df["Date"].dt.month,
        DayOfMonth=lambda df: df["Date"].dt.day,
        DayOfYear=lambda df: df["Date"].dt.day_of_year,
        DayOfWeek=lambda df: df["Date"].dt.day_of_week + 1,
        QuarterOfYear=lambda df: df["Date"].dt.quarter,
        IsWeekend=lambda df: df["Date"].dt.dayofweek.isin([5, 6]).astype(int),
        IsWeekday=lambda df: (~df["Date"].dt.dayofweek.isin([5, 6])).astype(int),
    ).drop(columns=["Date"])


def build_lag_features(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    tmp: pd.DataFrame = df.copy()
    lags: list[int] = tmp.index.to_list()
    return tmp.assign(**{f"Lag_{lag}": tmp[target_col].shift(lag) for lag in lags if lag != 0})

#### One at a Time

#### No code

#### Show code

In [ ]:
### Do fill, one record at a time ----

# Partially create RandomForestRegressor
RFR: partial[RandomForestRegressor] = partial(
    RandomForestRegressor,
    n_estimators=1000,
    n_jobs=-1,
    random_state=RANDOM_SEED,
)

# Reassign dataframe
data_algorithmic_1: pd.DataFrame = data.copy().assign(Fill=data["Missing"])

# Identify indexes of missing values
indexes_of_missing: list[int] = data_algorithmic_1[data_algorithmic_1["Fill"].isna()].index.to_list()

# Iterate over indexes of missing values
for idx in tqdm(indexes_of_missing):

    tmp_df: pd.DataFrame = (
        # Assign temporary dataframe
        data_algorithmic_1.copy()
        # Drop unnecessary columns
        .drop(columns=["index", "Value", "Missing"])
        # Filter table up until the index of the target missing value
        .iloc[: idx + 1]
        # Build temporal features
        .pipe(build_temporal_features)
        # Assign lag features
        .pipe(build_lag_features, target_col="Fill")
    )

    # Split data into train and test
    data_trn_X: np.ndarray = tmp_df.drop(columns=["Fill"]).iloc[:-1].values
    data_trn_y: np.ndarray = tmp_df[["Fill"]].iloc[:-1].values
    data_tst_X: np.ndarray = tmp_df.drop(columns=["Fill"]).iloc[-1:].values

    # Instantiate, fit, predict model
    pred: float = RFR().fit(data_trn_X, data_trn_y).predict(data_tst_X)[0]

    # Assign prediction
    data_algorithmic_1.loc[idx, "Fill"] = pred

In [ ]:
### Plot data ----
score_algorithmic_1: float = mape(data_algorithmic_1["Value"], data_algorithmic_1["Fill"]) * 100
plot_data(
    data=data_algorithmic_1,
    date_col="Date",
    missing_col="Missing",
    fill_col="Fill",
    title="Filling using Machine Learning (Random Forest Regression)",
    subtitle=f"MAPE={score_algorithmic_1:.2f}%",
    output_file="./images/06_filling_using_machine_learning_1.html",
)

#### All at Once

#### No code

#### Show code

In [ ]:
### Do fill, all at once ----

# Reassign dataframe
data_algorithmic_2: pd.DataFrame = data.copy().assign(Fill=data["Missing"])

# Identify indexes of missing values
indexes_of_missing: list[int] = data_algorithmic_2[data_algorithmic_2["Fill"].isna()].index.to_list()
indexes_of_existing: list[int] = data_algorithmic_2[data_algorithmic_2["Fill"].notna()].index.to_list()

# Partially create RandomForestRegressor
RFR: partial[RandomForestRegressor] = partial(
    RandomForestRegressor,
    n_estimators=1000,
    n_jobs=-1,
    random_state=RANDOM_SEED,
)

tmp_df: pd.DataFrame = (
    # Assign temporary dataframe
    data_algorithmic_2.copy()
    # Drop unnecessary columns
    .drop(columns=["index", "Value", "Missing"])
    # Build temporal features
    .pipe(build_temporal_features)
    # Assign lag features
    .pipe(build_lag_features, target_col="Fill")
)

# Split data into train and test
data_trn_X: np.ndarray = tmp_df.drop(columns=["Fill"]).iloc[indexes_of_existing, :].values
data_trn_y: np.ndarray = tmp_df.loc[indexes_of_existing, ["Fill"]].values
data_tst_X: np.ndarray = tmp_df.drop(columns=["Fill"]).iloc[indexes_of_missing, :].values

# Instantiate, fit, predict model
pred: np.ndarray = RFR().fit(data_trn_X, data_trn_y).predict(data_tst_X)

# Assign prediction
data_algorithmic_2.loc[indexes_of_missing, "Fill"] = pred

In [ ]:
### Plot data ----
score_algorithmic_2: float = mape(data_algorithmic_2["Value"], data_algorithmic_2["Fill"]) * 100
plot_data(
    data=data_algorithmic_2,
    date_col="Date",
    missing_col="Missing",
    fill_col="Fill",
    title="Filling using Machine Learning (Random Forest Regression)",
    subtitle=f"MAPE={score_algorithmic_2:.2f}%",
    output_file="./images/06_filling_using_machine_learning_2.html",
)

### Embedding

Embedding methods are primarily used to remove noise and focus on the main information in the data. It can also be used to fill gaps just like de-noising, once the embedding has been identified. Can use generalised models like GLRM, or even autoencoders like MIDAS.

Advantages:

- Strong for non-trivial, non-linear patterns
- In particular when combined with multiple imputations

Disadvantages:

- Not very established method (compared to others mentioned)
- Have to use specific packages to implement

When to use:

- You look for a best-in-class solution for filling missing values

<table>
    <tr>
        <td>
            <div class="center"><b>Source</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td>&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td>&nbsp;</td><td>&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td>&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
        <td>
            <div class="center">Something</b><br>➡️</div>
        </td>
        <td>
            <div class="center"><b>Result</b></div>
            <table>
                <tr><th>t</th><th>y</th><th>x1</th><th>x2</th></tr>
                <tr><td>2023-01-01</td><td>9.8</td><td class="green">&nbsp;</td><td>1.2</td></tr>
                <tr><td>2023-01-02</td><td>10.3</td><td>18.6</td><td>2.3</td></tr>
                <tr><td>2023-01-03</td><td>24.6</td><td>49.2</td><td>0.5</td></tr>
                <tr><td>2023-01-04</td><td>7.5</td><td>14.4</td><td>0.7</td></tr>
                <tr><td>2023-01-05</td><td>11.1</td><td class="green">&nbsp;</td><td class="green">&nbsp;</td></tr>
                <tr><td>2023-01-06</td><td>10.0</td><td>18.9</td><td>1.1</td></tr>
                <tr><td>2023-01-07</td><td>9.9</td><td>19.1</td><td>0.8</td></tr>
                <tr><td>2023-01-08</td><td>8.7</td><td class="green">&nbsp;</td><td>0.8</td></tr>
                <tr><td>2023-01-09</td><td>10.6</td><td>20.1</td><td>0.9</td></tr>
                <tr><td>2023-01-10</td><td>11.4</td><td>21.9</td><td>1.0</td></tr>
            </table>
        </td>
    </tr>
</table>

## Summary

[^munchhausen-trilemma]: The Münchhausen trilemma asserts that there are only three ways of completing a proof; by circular argument, regressive argument, and dogmatic argument. Baron Münchhausen proposed a thought experiment where he tried to prove it was theoretically possible to free himself out of being stuck in the mud by pulling himself out with his own hair.

[`numpy]: https://numpy.org/
[`pandas`]: https://pandas.pydata.org/
[`synthetic_data_generators`]: https://data-science-extensions.com/toolboxes/synthetic-data-generators/
[`plotly`]: https://plotly.com/python/
[`pmdarima`]: https://alkaline-ml.com/pmdarima/
[`sklearn`]: https://scikit-learn.org/
[`tqdm`]: https://tqdm.github.io/
